In [13]:
import os
import logging
from pinecone import Pinecone
from langchain_pinecone import Pinecone as PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

# Load environment variables
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_INDEX_NAME = "movies-actors"

# Initialize Pinecone client
logging.info("Initializing Pinecone client...")
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

# Initialize embedding model
logging.info("Initializing embeddings model...")
embeddings_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

def retrieve_single_row(query_text: str, top_k: int = 5):
    """
    Retrieves a single row (or top-k results) from Pinecone based on a query.
    """
    # Convert query text to embedding
    query_embedding = embeddings_model.embed_query(query_text)
    
    # Search Pinecone index
    logging.info(f"Searching Pinecone for query: {query_text}")
    search_results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    # Extract and return results
    if search_results and "matches" in search_results and search_results["matches"]:
        best_match = search_results["matches"][0:top_k]  # Retrieve the highest-ranked match
        logging.info(f"Best match: {best_match}")
        return best_match
    else:
        logging.warning("No matching results found.")
        return None

# Example usage
if __name__ == "__main__":
    query = "An emotional drama about a killer"
    result = retrieve_single_row(query)
    print("Retrieved Row:", result)

Retrieved Row: [{'id': 'tt29265463',
 'metadata': {'actor': 'Chuck Allen,Malachi E. Hamilton,Jay Harmon III,Brandon '
                       'Hickman,Fracaswell Hyman,Jay James,Prince James,Brad '
                       'Elle Jones',
              'actress': 'Rese Cheri,Aimee Kochan',
              'averageRating': '6.0',
              'director': 'Nakia T Hamilton',
              'endYear': '',
              'genres': 'Drama',
              'isAdult': 'True',
              'numVotes': '37.0',
              'originalTitle': 'Emotions of a Murder',
              'primaryTitle': 'Emotions of a Murder',
              'producer': 'Malachi E. Hamilton',
              'runtimeMinutes': '80',
              'startYear': '2024.0',
              'tconst': 'tt29265463',
              'titleType': 'movie',
              'writer': 'Nakia T Hamilton'},
 'score': 0.81708318,
 'values': []}, {'id': 'tt0833484',
 'metadata': {'actor': 'Irrfan Khan,Emraan Hashmi,Sanjay Batra,Amit '
                     

In [ ]:
import os
from PineconeDataUpload import get_vector_store

def test_vector_store(query: str, k: int = 5):
    """Fetches movie recommendations from the vector store based on a query."""
    
    # Initialize vector store
    print("Initializing Vector Store...")
    vector_store = get_vector_store()
    
    # Perform similarity search
    print(f"Querying Vector Store with: {query}")
    retrieved_docs = vector_store.similarity_search(query, k=k)

    # Print raw results before processing
    print("\nRAW Retrieved Data (Before LangChain Processing):")
    print(retrieved_docs)

    # Display structured document info
    print("\nProcessed Retrieved Documents:")
    for i, doc in enumerate(retrieved_docs, 1):
        print(f"\nMovie {i}:")
        print(f"Content: {doc.page_content if hasattr(doc, 'page_content') else 'MISSING page_content'}")
        print(f"Metadata: {doc.metadata if hasattr(doc, 'metadata') else 'No metadata available'}")

# Example Query
if __name__ == "__main__":
    user_query = "Recommend a sci-fi movie from the last decade"
    test_vector_store(user_query, k=5)

Initializing Vector Store...
Querying Vector Store with: Recommend a sci-fi movie from the last decade



RAW Retrieved Data (Before LangChain Processing):
[]

Processed Retrieved Documents:


In [12]:
def retrieve_with_langchain(query: str, k: int = 5):
    """Retrieves relevant documents using LangChain's Pinecone integration."""
    
    # Initialize vector store
    logging.info("Initializing LangChain Vector Store...")
    vector_store = PineconeVectorStore(index, embeddings_model)

    # Retrieve documents
    logging.info(f"Querying LangChain Pinecone Vector Store for: {query}")
    retrieved_docs = vector_store.similarity_search(query, k=k)

    # Print retrieved documents
    print("\nRetrieved Documents via LangChain:")
    for i, doc in enumerate(retrieved_docs, 1):
        print(f"\nMovie {i}:")
        print(f"Content: {doc.page_content if hasattr(doc, 'page_content') else 'MISSING page_content'}")
        print(f"Metadata: {doc.metadata if hasattr(doc, 'metadata') else 'No metadata available'}")

# Run test
if __name__ == "__main__":
    query_text = "An emotional drama about a killer"
    retrieve_with_langchain(query_text, k=5)


Retrieved Documents via LangChain:
